In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model
from sklearn.preprocessing import MinMaxScaler

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt

In [2]:
# Load the features from the CSV file
df = pd.read_csv(r'Original_Features6.csv')
df=df.drop(['rolloff','zero_crossing_rate','chroma_stft','spectral_centroid','spectral_bandwidth'],axis=1)
df

,mfcc_1,mfcc_2,mfcc_3,mfcc_4,mfcc_5,mfcc_6,mfcc_7,mfcc_8,mfcc_9,mfcc_10,mfcc_11,mfcc_12,mfcc_13,English Type
0,-351.79230,147.745390,19.917744,31.775644,23.516031,2.070087,9.844380,0.265172,-2.338450,-1.322878,-9.483175,-1.679585,-0.177087,0
1,-410.52670,125.953590,-20.461075,17.156311,-6.669661,15.239104,-16.752926,-3.614888,-5.258536,-7.771095,-2.448791,-5.192124,-2.269287,0
2,-342.62964,78.433655,12.540177,31.937393,-2.213818,15.747808,-39.245610,-5.218530,-16.471539,-5.059716,-14.047500,-2.351587,-14.914782,0
3,-331.50244,108.531410,-21.552880,42.042065,-4.467168,9.407474,1.117538,-26.776285,-9.033010,-18.285458,5.542378,1.801108,-12.343457,1
4,-306.30480,105.473820,-2.033697,33.645050,19.767216,14.690701,-7.823933,16.233910,1.511278,-1.097624,-4.256559,5.822567,-5.417769,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2133,-367.42440,153.012450,-6.412210,19.019106,17.933186,-5.957310,-0.477651,-18.703012,-15.582651,-1.840943,-12.552829,-8.802483,-3.864208,1
2134,-321.66678,128.274500,-3.335473,28.203968,15.083783,16.396720,4.090911,10.306094,2.037651,0.225633,1.150143,1.580174,-2.241047,0
2135,-253.21036,93.182236,9.850489,38.595917,-14.542635,16.471262,-13.102000,20.879627,-8.502631,6.002625,12.753398,2.869624,-1.761060,0
2136,-322.13104,150.674900,-11.696100,24.301128,10.992788,-7.354234,20.903921,-4.221878,-11.789002,8.365523,-4.948959,-10.440193,-3.944272,0


In [3]:
X = df.iloc[:,0:13]
y = df.iloc[:,13]


In [4]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(X_train.shape,y_train.shape)

# scale data
t = MinMaxScaler()
t.fit(X_train)
X_train = t.transform(X_train)
X_test = t.transform(X_test)


(1710, 13) (1710,)


In [5]:
# Standardize the input features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [6]:
# Define the autoencoder architecture
input_dim = X_train_scaled.shape[1]
encoding_dim = 64
input_layer = Input(shape=(input_dim,))
encoder = Dense(encoding_dim, activation='relu')(input_layer)
decoder = Dense(input_dim, activation='sigmoid')(encoder)
autoencoder = Model(inputs=input_layer, outputs=decoder)
autoencoder.compile(optimizer='rmsprop', loss='mse')

# Train the autoencoder
autoencoder.fit(X_train_scaled, X_train_scaled, epochs=1500, batch_size=40,
                validation_data=(X_test_scaled, X_test_scaled))

# Extract the encoded representations
encoder = Model(inputs=input_layer, outputs=encoder)
X_train_encoded = encoder.predict(X_train_scaled)
X_test_encoded = encoder.predict(X_test_scaled)


Epoch 1/1500
43/43 [==============================] - 1s 10ms/step - loss: 1.0976 - val_loss: 1.0150
Epoch 2/1500
43/43 [==============================] - 0s 4ms/step - loss: 0.9112 - val_loss: 0.8754
Epoch 3/1500
43/43 [==============================] - 0s 4ms/step - loss: 0.8058 - val_loss: 0.7992
Epoch 4/1500
43/43 [==============================] - 0s 3ms/step - loss: 0.7466 - val_loss: 0.7533
Epoch 5/1500
43/43 [==============================] - 0s 4ms/step - loss: 0.7089 - val_loss: 0.7221
Epoch 6/1500
43/43 [==============================] - 0s 3ms/step - loss: 0.6816 - val_loss: 0.6981
Epoch 7/1500
43/43 [==============================] - 0s 4ms/step - loss: 0.6608 - val_loss: 0.6796
Epoch 8/1500
43/43 [==============================] - 0s 4ms/step - loss: 0.6448 - val_loss: 0.6646
Epoch 9/1500
43/43 [==============================] - 0s 4ms/step - loss: 0.6325 - val_loss: 0.6529
Epoch 10/1500
43/43 [==============================] - 0s 4ms/step - loss: 0.6229 - val_loss: 0.643

43/43 [==============================] - 0s 3ms/step - loss: 0.5857 - val_loss: 0.6076
Epoch 83/1500
43/43 [==============================] - 0s 4ms/step - loss: 0.5857 - val_loss: 0.6076
Epoch 84/1500
43/43 [==============================] - 0s 3ms/step - loss: 0.5857 - val_loss: 0.6076
Epoch 85/1500
43/43 [==============================] - 0s 4ms/step - loss: 0.5857 - val_loss: 0.6076
Epoch 86/1500
43/43 [==============================] - 0s 3ms/step - loss: 0.5857 - val_loss: 0.6076
Epoch 87/1500
43/43 [==============================] - 0s 4ms/step - loss: 0.5857 - val_loss: 0.6076
Epoch 88/1500
43/43 [==============================] - 0s 3ms/step - loss: 0.5856 - val_loss: 0.6075
Epoch 89/1500
43/43 [==============================] - 0s 3ms/step - loss: 0.5856 - val_loss: 0.6075
Epoch 90/1500
43/43 [==============================] - 0s 4ms/step - loss: 0.5856 - val_loss: 0.6075
Epoch 91/1500
43/43 [==============================] - 0s 3ms/step - loss: 0.5856 - val_loss: 0.6075
Epoc

43/43 [==============================] - 0s 4ms/step - loss: 0.5852 - val_loss: 0.6070
Epoch 163/1500
43/43 [==============================] - 0s 3ms/step - loss: 0.5852 - val_loss: 0.6070
Epoch 164/1500
43/43 [==============================] - 0s 4ms/step - loss: 0.5852 - val_loss: 0.6070
Epoch 165/1500
43/43 [==============================] - 0s 3ms/step - loss: 0.5852 - val_loss: 0.6070
Epoch 166/1500
43/43 [==============================] - 0s 4ms/step - loss: 0.5852 - val_loss: 0.6070
Epoch 167/1500
43/43 [==============================] - 0s 3ms/step - loss: 0.5852 - val_loss: 0.6070
Epoch 168/1500
43/43 [==============================] - 0s 3ms/step - loss: 0.5852 - val_loss: 0.6070
Epoch 169/1500
43/43 [==============================] - 0s 4ms/step - loss: 0.5852 - val_loss: 0.6070
Epoch 170/1500
43/43 [==============================] - 0s 3ms/step - loss: 0.5852 - val_loss: 0.6070
Epoch 171/1500
43/43 [==============================] - 0s 4ms/step - loss: 0.5852 - val_loss: 0.

43/43 [==============================] - 0s 4ms/step - loss: 0.5850 - val_loss: 0.6068
Epoch 243/1500
43/43 [==============================] - 0s 3ms/step - loss: 0.5850 - val_loss: 0.6068
Epoch 244/1500
43/43 [==============================] - 0s 4ms/step - loss: 0.5850 - val_loss: 0.6068
Epoch 245/1500
43/43 [==============================] - 0s 3ms/step - loss: 0.5850 - val_loss: 0.6068
Epoch 246/1500
43/43 [==============================] - 0s 4ms/step - loss: 0.5850 - val_loss: 0.6068
Epoch 247/1500
43/43 [==============================] - 0s 3ms/step - loss: 0.5850 - val_loss: 0.6068
Epoch 248/1500
43/43 [==============================] - 0s 4ms/step - loss: 0.5850 - val_loss: 0.6068
Epoch 249/1500
43/43 [==============================] - 0s 3ms/step - loss: 0.5850 - val_loss: 0.6068
Epoch 250/1500
43/43 [==============================] - 0s 4ms/step - loss: 0.5850 - val_loss: 0.6068
Epoch 251/1500
43/43 [==============================] - 0s 4ms/step - loss: 0.5850 - val_loss: 0.

43/43 [==============================] - 0s 4ms/step - loss: 0.5848 - val_loss: 0.6067
Epoch 323/1500
43/43 [==============================] - 0s 3ms/step - loss: 0.5848 - val_loss: 0.6067
Epoch 324/1500
43/43 [==============================] - 0s 3ms/step - loss: 0.5848 - val_loss: 0.6066
Epoch 325/1500
43/43 [==============================] - 0s 4ms/step - loss: 0.5848 - val_loss: 0.6067
Epoch 326/1500
43/43 [==============================] - 0s 4ms/step - loss: 0.5848 - val_loss: 0.6067
Epoch 327/1500
43/43 [==============================] - 0s 4ms/step - loss: 0.5848 - val_loss: 0.6067
Epoch 328/1500
43/43 [==============================] - 0s 3ms/step - loss: 0.5848 - val_loss: 0.6067
Epoch 329/1500
43/43 [==============================] - 0s 4ms/step - loss: 0.5848 - val_loss: 0.6067
Epoch 330/1500
43/43 [==============================] - 0s 3ms/step - loss: 0.5848 - val_loss: 0.6067
Epoch 331/1500
43/43 [==============================] - 0s 4ms/step - loss: 0.5848 - val_loss: 0.

...

43/43 [==============================] - 0s 3ms/step - loss: 0.5845 - val_loss: 0.6063
Epoch 1122/1500
43/43 [==============================] - 0s 4ms/step - loss: 0.5845 - val_loss: 0.6063
Epoch 1123/1500
43/43 [==============================] - 0s 3ms/step - loss: 0.5845 - val_loss: 0.6063
Epoch 1124/1500
43/43 [==============================] - 0s 4ms/step - loss: 0.5845 - val_loss: 0.6063
Epoch 1125/1500
43/43 [==============================] - 0s 3ms/step - loss: 0.5845 - val_loss: 0.6063
Epoch 1126/1500
43/43 [==============================] - 0s 4ms/step - loss: 0.5845 - val_loss: 0.6063
Epoch 1127/1500
43/43 [==============================] - 0s 3ms/step - loss: 0.5845 - val_loss: 0.6063
Epoch 1128/1500
43/43 [==============================] - 0s 3ms/step - loss: 0.5845 - val_loss: 0.6063
Epoch 1129/1500
43/43 [==============================] - 0s 4ms/step - loss: 0.5845 - val_loss: 0.6063
Epoch 1130/1500
43/43 [==============================] - 0s 3ms/step - loss: 0.5845 - val

43/43 [==============================] - 0s 3ms/step - loss: 0.5845 - val_loss: 0.6063
Epoch 1201/1500
43/43 [==============================] - 0s 3ms/step - loss: 0.5845 - val_loss: 0.6063
Epoch 1202/1500
43/43 [==============================] - 0s 3ms/step - loss: 0.5845 - val_loss: 0.6063
Epoch 1203/1500
43/43 [==============================] - 0s 3ms/step - loss: 0.5845 - val_loss: 0.6063
Epoch 1204/1500
43/43 [==============================] - 0s 3ms/step - loss: 0.5845 - val_loss: 0.6064
Epoch 1205/1500
43/43 [==============================] - 0s 3ms/step - loss: 0.5845 - val_loss: 0.6063
Epoch 1206/1500
43/43 [==============================] - 0s 3ms/step - loss: 0.5845 - val_loss: 0.6063
Epoch 1207/1500
43/43 [==============================] - 0s 3ms/step - loss: 0.5845 - val_loss: 0.6063
Epoch 1208/1500
43/43 [==============================] - 0s 3ms/step - loss: 0.5845 - val_loss: 0.6063
Epoch 1209/1500
43/43 [==============================] - 0s 3ms/step - loss: 0.5845 - val

43/43 [==============================] - 0s 3ms/step - loss: 0.5845 - val_loss: 0.6063
Epoch 1280/1500
43/43 [==============================] - 0s 3ms/step - loss: 0.5845 - val_loss: 0.6063
Epoch 1281/1500
43/43 [==============================] - 0s 3ms/step - loss: 0.5845 - val_loss: 0.6063
Epoch 1282/1500
43/43 [==============================] - 0s 3ms/step - loss: 0.5845 - val_loss: 0.6063
Epoch 1283/1500
43/43 [==============================] - 0s 3ms/step - loss: 0.5845 - val_loss: 0.6063
Epoch 1284/1500
43/43 [==============================] - 0s 3ms/step - loss: 0.5845 - val_loss: 0.6063
Epoch 1285/1500
43/43 [==============================] - 0s 3ms/step - loss: 0.5845 - val_loss: 0.6063
Epoch 1286/1500
43/43 [==============================] - 0s 3ms/step - loss: 0.5845 - val_loss: 0.6064
Epoch 1287/1500
43/43 [==============================] - 0s 3ms/step - loss: 0.5845 - val_loss: 0.6063
Epoch 1288/1500
43/43 [==============================] - 0s 3ms/step - loss: 0.5845 - val

43/43 [==============================] - 0s 3ms/step - loss: 0.5845 - val_loss: 0.6063
Epoch 1359/1500
43/43 [==============================] - 0s 3ms/step - loss: 0.5845 - val_loss: 0.6063
Epoch 1360/1500
43/43 [==============================] - 0s 3ms/step - loss: 0.5845 - val_loss: 0.6063
Epoch 1361/1500
43/43 [==============================] - 0s 3ms/step - loss: 0.5845 - val_loss: 0.6063
Epoch 1362/1500
43/43 [==============================] - 0s 3ms/step - loss: 0.5845 - val_loss: 0.6063
Epoch 1363/1500
43/43 [==============================] - 0s 3ms/step - loss: 0.5845 - val_loss: 0.6063
Epoch 1364/1500
43/43 [==============================] - 0s 3ms/step - loss: 0.5845 - val_loss: 0.6063
Epoch 1365/1500
43/43 [==============================] - 0s 3ms/step - loss: 0.5845 - val_loss: 0.6063
Epoch 1366/1500
43/43 [==============================] - 0s 3ms/step - loss: 0.5845 - val_loss: 0.6063
Epoch 1367/1500
43/43 [==============================] - 0s 3ms/step - loss: 0.5845 - val

43/43 [==============================] - 0s 3ms/step - loss: 0.5845 - val_loss: 0.6063
Epoch 1438/1500
43/43 [==============================] - 0s 3ms/step - loss: 0.5845 - val_loss: 0.6063
Epoch 1439/1500
43/43 [==============================] - 0s 3ms/step - loss: 0.5845 - val_loss: 0.6063
Epoch 1440/1500
43/43 [==============================] - 0s 3ms/step - loss: 0.5845 - val_loss: 0.6063
Epoch 1441/1500
43/43 [==============================] - 0s 3ms/step - loss: 0.5845 - val_loss: 0.6063
Epoch 1442/1500
43/43 [==============================] - 0s 3ms/step - loss: 0.5845 - val_loss: 0.6063
Epoch 1443/1500
43/43 [==============================] - 0s 3ms/step - loss: 0.5845 - val_loss: 0.6063
Epoch 1444/1500
43/43 [==============================] - 0s 3ms/step - loss: 0.5845 - val_loss: 0.6063
Epoch 1445/1500
43/43 [==============================] - 0s 3ms/step - loss: 0.5845 - val_loss: 0.6063
Epoch 1446/1500
43/43 [==============================] - 0s 3ms/step - loss: 0.5845 - val

In [7]:
#Train a classifier
from sklearn.svm import SVC
clf = SVC()
clf.fit(X_train_encoded, y_train)

# Evaluate the performance
y_pred = clf.predict(X_test_encoded)
print("Accuracy: ", accuracy_score(y_test, y_pred))
print("Precision: ", precision_score(y_test, y_pred, average='weighted'))
print("Recall: ", recall_score(y_test, y_pred, average='weighted'))
print("F1-score: ", f1_score(y_test, y_pred, average='weighted'))


import matplotlib.pyplot as plt
from sklearn.metrics import plot_confusion_matrix, classification_report
import warnings

# Print classification report
print(classification_report(y_test, y_pred))

Accuracy:  0.8177570093457944
Precision:  0.6687265263341776
Recall:  0.8177570093457944
F1-score:  0.7357710881003291
              precision    recall  f1-score   support

           0       0.82      1.00      0.90       350
           1       0.00      0.00      0.00        78

    accuracy                           0.82       428
   macro avg       0.41      0.50      0.45       428
weighted avg       0.67      0.82      0.74       428



In [8]:
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score


model = XGBClassifier()
history = model.fit(X_train_encoded, y_train)
#print(model)

y_pred = model.predict(X_test_encoded)
predictions = [round(value) for value in y_pred]

# evaluate predictions
accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))


Accuracy: 79.91%


In [9]:
import matplotlib.pyplot as plt
from sklearn.metrics import plot_confusion_matrix, classification_report
import warnings

# Print classification report
print(classification_report(y_test, y_pred))



              precision    recall  f1-score   support

           0       0.82      0.96      0.89       350
           1       0.28      0.06      0.10        78

    accuracy                           0.80       428
   macro avg       0.55      0.51      0.50       428
weighted avg       0.72      0.80      0.74       428



# CNN

In [10]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout
from keras.callbacks import EarlyStopping

In [11]:
X = df.iloc[:,0:13]
y = df.iloc[:,13]

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)


In [12]:
# Convert the accent labels to integers
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

# Convert the accent labels to one-hot vectors
num_classes = len(np.unique(y_train_encoded))
y_train_onehot = to_categorical(y_train_encoded, num_classes)
y_test_onehot = to_categorical(y_test_encoded, num_classes)

# Reshape the input features for the CNN
n_timesteps, n_features = X_train.shape[1], 1
X_train_reshaped = X_train.values.reshape((X_train.shape[0], n_timesteps, n_features))
X_test_reshaped = X_test.values.reshape((X_test.shape[0], n_timesteps, n_features))


In [13]:
# Define the CNN architecture
model = Sequential()
model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(n_timesteps, n_features)))
model.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
model.add(Dropout(0.5))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(100, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])



In [ ]:
# Train the CNN
#es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10)
history = model.fit(X_train_reshaped, y_train_onehot, epochs=1500, batch_size=40,
                    validation_data=(X_test_reshaped, y_test_onehot))


Epoch 1/1500
38/38 [==============================] - 1s 8ms/step - loss: 1.1983 - accuracy: 0.7273 - val_loss: 0.6197 - val_accuracy: 0.8131
Epoch 2/1500
38/38 [==============================] - 0s 4ms/step - loss: 0.5941 - accuracy: 0.7921 - val_loss: 0.4796 - val_accuracy: 0.8115
Epoch 3/1500
38/38 [==============================] - 0s 4ms/step - loss: 0.5135 - accuracy: 0.8021 - val_loss: 0.5646 - val_accuracy: 0.8131
Epoch 4/1500
38/38 [==============================] - 0s 5ms/step - loss: 0.4855 - accuracy: 0.8015 - val_loss: 0.4884 - val_accuracy: 0.8131
Epoch 5/1500
38/38 [==============================] - 0s 5ms/step - loss: 0.4730 - accuracy: 0.8195 - val_loss: 0.4930 - val_accuracy: 0.8131
Epoch 6/1500
38/38 [==============================] - 0s 5ms/step - loss: 0.4621 - accuracy: 0.8189 - val_loss: 0.4971 - val_accuracy: 0.8100
Epoch 7/1500
38/38 [==============================] - 0s 4ms/step - loss: 0.4445 - accuracy: 0.8302 - val_loss: 0.4737 - val_accuracy: 0.8115
Epoch 

38/38 [==============================] - 0s 4ms/step - loss: 0.3751 - accuracy: 0.8463 - val_loss: 0.4765 - val_accuracy: 0.8100
Epoch 59/1500
38/38 [==============================] - 0s 4ms/step - loss: 0.3816 - accuracy: 0.8382 - val_loss: 0.5020 - val_accuracy: 0.8115
Epoch 60/1500
38/38 [==============================] - 0s 4ms/step - loss: 0.3765 - accuracy: 0.8436 - val_loss: 0.4840 - val_accuracy: 0.8162
Epoch 61/1500
38/38 [==============================] - 0s 4ms/step - loss: 0.3736 - accuracy: 0.8429 - val_loss: 0.5074 - val_accuracy: 0.8146
Epoch 62/1500
38/38 [==============================] - 0s 4ms/step - loss: 0.3808 - accuracy: 0.8416 - val_loss: 0.5299 - val_accuracy: 0.8084
Epoch 63/1500
38/38 [==============================] - 0s 5ms/step - loss: 0.3828 - accuracy: 0.8422 - val_loss: 0.4846 - val_accuracy: 0.8146
Epoch 64/1500
38/38 [==============================] - 0s 5ms/step - loss: 0.3711 - accuracy: 0.8489 - val_loss: 0.4860 - val_accuracy: 0.8146
Epoch 65/1500

38/38 [==============================] - 0s 4ms/step - loss: 0.3370 - accuracy: 0.8590 - val_loss: 0.5496 - val_accuracy: 0.8084
Epoch 116/1500
38/38 [==============================] - 0s 5ms/step - loss: 0.3198 - accuracy: 0.8717 - val_loss: 0.5598 - val_accuracy: 0.8115
Epoch 117/1500
38/38 [==============================] - 0s 5ms/step - loss: 0.3396 - accuracy: 0.8570 - val_loss: 0.5191 - val_accuracy: 0.8006
Epoch 118/1500
38/38 [==============================] - 0s 4ms/step - loss: 0.3423 - accuracy: 0.8576 - val_loss: 0.5407 - val_accuracy: 0.8131
Epoch 119/1500
38/38 [==============================] - 0s 4ms/step - loss: 0.3330 - accuracy: 0.8690 - val_loss: 0.5530 - val_accuracy: 0.8146
Epoch 120/1500
38/38 [==============================] - 0s 4ms/step - loss: 0.3315 - accuracy: 0.8603 - val_loss: 0.5012 - val_accuracy: 0.8084
Epoch 121/1500
38/38 [==============================] - 0s 4ms/step - loss: 0.3305 - accuracy: 0.8656 - val_loss: 0.5722 - val_accuracy: 0.8100
Epoch 1

Epoch 172/1500
38/38 [==============================] - 0s 4ms/step - loss: 0.2993 - accuracy: 0.8770 - val_loss: 0.5937 - val_accuracy: 0.8100
Epoch 173/1500
38/38 [==============================] - 0s 4ms/step - loss: 0.3023 - accuracy: 0.8790 - val_loss: 0.5334 - val_accuracy: 0.8084
Epoch 174/1500
38/38 [==============================] - 0s 4ms/step - loss: 0.2984 - accuracy: 0.8770 - val_loss: 0.5480 - val_accuracy: 0.8100
Epoch 175/1500
38/38 [==============================] - 0s 4ms/step - loss: 0.2870 - accuracy: 0.8810 - val_loss: 0.5894 - val_accuracy: 0.8022
Epoch 176/1500
38/38 [==============================] - 0s 4ms/step - loss: 0.3098 - accuracy: 0.8723 - val_loss: 0.5762 - val_accuracy: 0.8146
Epoch 177/1500
38/38 [==============================] - 0s 4ms/step - loss: 0.2854 - accuracy: 0.8857 - val_loss: 0.5793 - val_accuracy: 0.8069
Epoch 178/1500
38/38 [==============================] - 0s 4ms/step - loss: 0.2935 - accuracy: 0.8730 - val_loss: 0.5749 - val_accuracy:

Epoch 229/1500
38/38 [==============================] - 0s 4ms/step - loss: 0.2689 - accuracy: 0.8810 - val_loss: 0.6188 - val_accuracy: 0.8053
Epoch 230/1500
38/38 [==============================] - 0s 4ms/step - loss: 0.2691 - accuracy: 0.8897 - val_loss: 0.6137 - val_accuracy: 0.8069
Epoch 231/1500
38/38 [==============================] - 0s 4ms/step - loss: 0.2744 - accuracy: 0.8850 - val_loss: 0.6029 - val_accuracy: 0.8084
Epoch 232/1500
38/38 [==============================] - 0s 4ms/step - loss: 0.2593 - accuracy: 0.8884 - val_loss: 0.6342 - val_accuracy: 0.8069
Epoch 233/1500
38/38 [==============================] - 0s 4ms/step - loss: 0.2673 - accuracy: 0.8951 - val_loss: 0.6135 - val_accuracy: 0.8069
Epoch 234/1500
38/38 [==============================] - 0s 4ms/step - loss: 0.2721 - accuracy: 0.8824 - val_loss: 0.5615 - val_accuracy: 0.8022
Epoch 235/1500
38/38 [==============================] - 0s 4ms/step - loss: 0.2559 - accuracy: 0.8924 - val_loss: 0.5679 - val_accuracy:

...

38/38 [==============================] - 0s 4ms/step - loss: 0.1182 - accuracy: 0.9519 - val_loss: 1.2061 - val_accuracy: 0.7804
Epoch 1084/1500
38/38 [==============================] - 0s 4ms/step - loss: 0.0987 - accuracy: 0.9592 - val_loss: 1.4428 - val_accuracy: 0.7913
Epoch 1085/1500
38/38 [==============================] - 0s 4ms/step - loss: 0.1080 - accuracy: 0.9666 - val_loss: 1.6771 - val_accuracy: 0.7882
Epoch 1086/1500
38/38 [==============================] - 0s 4ms/step - loss: 0.1104 - accuracy: 0.9639 - val_loss: 1.4066 - val_accuracy: 0.7928
Epoch 1087/1500
38/38 [==============================] - 0s 4ms/step - loss: 0.0786 - accuracy: 0.9672 - val_loss: 1.4755 - val_accuracy: 0.7835
Epoch 1088/1500
38/38 [==============================] - 0s 4ms/step - loss: 0.1047 - accuracy: 0.9632 - val_loss: 1.3864 - val_accuracy: 0.7741
Epoch 1089/1500
38/38 [==============================] - 0s 4ms/step - loss: 0.1345 - accuracy: 0.9566 - val_loss: 1.3882 - val_accuracy: 0.7804
E

38/38 [==============================] - 0s 4ms/step - loss: 0.1047 - accuracy: 0.9599 - val_loss: 1.5141 - val_accuracy: 0.7991
Epoch 1140/1500
38/38 [==============================] - 0s 4ms/step - loss: 0.0910 - accuracy: 0.9753 - val_loss: 1.2807 - val_accuracy: 0.7664
Epoch 1141/1500
38/38 [==============================] - 0s 4ms/step - loss: 0.0953 - accuracy: 0.9599 - val_loss: 1.4042 - val_accuracy: 0.7819
Epoch 1142/1500
38/38 [==============================] - 0s 4ms/step - loss: 0.1022 - accuracy: 0.9592 - val_loss: 1.3909 - val_accuracy: 0.7835
Epoch 1143/1500
38/38 [==============================] - 0s 4ms/step - loss: 0.1178 - accuracy: 0.9639 - val_loss: 1.1936 - val_accuracy: 0.7617
Epoch 1144/1500
38/38 [==============================] - 0s 4ms/step - loss: 0.1104 - accuracy: 0.9579 - val_loss: 1.4691 - val_accuracy: 0.7913
Epoch 1145/1500
38/38 [==============================] - 0s 4ms/step - loss: 0.0982 - accuracy: 0.9632 - val_loss: 1.3810 - val_accuracy: 0.7757
E

38/38 [==============================] - 0s 4ms/step - loss: 0.1195 - accuracy: 0.9586 - val_loss: 1.4423 - val_accuracy: 0.7850
Epoch 1196/1500
38/38 [==============================] - 0s 4ms/step - loss: 0.0888 - accuracy: 0.9659 - val_loss: 1.7664 - val_accuracy: 0.7944
Epoch 1197/1500
38/38 [==============================] - 0s 4ms/step - loss: 0.1045 - accuracy: 0.9639 - val_loss: 1.7749 - val_accuracy: 0.7928
Epoch 1198/1500
38/38 [==============================] - 0s 4ms/step - loss: 0.0939 - accuracy: 0.9652 - val_loss: 1.4573 - val_accuracy: 0.7741
Epoch 1199/1500
38/38 [==============================] - 0s 4ms/step - loss: 0.0904 - accuracy: 0.9719 - val_loss: 1.6827 - val_accuracy: 0.7819
Epoch 1200/1500
38/38 [==============================] - 0s 4ms/step - loss: 0.1204 - accuracy: 0.9652 - val_loss: 1.6325 - val_accuracy: 0.7835
Epoch 1201/1500
38/38 [==============================] - 0s 4ms/step - loss: 0.0920 - accuracy: 0.9659 - val_loss: 1.4578 - val_accuracy: 0.7757
E

38/38 [==============================] - 0s 4ms/step - loss: 0.0946 - accuracy: 0.9659 - val_loss: 1.4352 - val_accuracy: 0.7928
Epoch 1252/1500
38/38 [==============================] - 0s 4ms/step - loss: 0.1257 - accuracy: 0.9612 - val_loss: 1.4248 - val_accuracy: 0.7788
Epoch 1253/1500
38/38 [==============================] - 0s 4ms/step - loss: 0.1236 - accuracy: 0.9566 - val_loss: 1.5275 - val_accuracy: 0.7928
Epoch 1254/1500
38/38 [==============================] - 0s 4ms/step - loss: 0.0914 - accuracy: 0.9693 - val_loss: 1.4353 - val_accuracy: 0.7773
Epoch 1255/1500
38/38 [==============================] - 0s 4ms/step - loss: 0.0817 - accuracy: 0.9726 - val_loss: 1.6078 - val_accuracy: 0.7804
Epoch 1256/1500
38/38 [==============================] - 0s 4ms/step - loss: 0.1053 - accuracy: 0.9632 - val_loss: 1.2395 - val_accuracy: 0.7508
Epoch 1257/1500
38/38 [==============================] - 0s 4ms/step - loss: 0.0864 - accuracy: 0.9686 - val_loss: 1.5273 - val_accuracy: 0.7835
E

38/38 [==============================] - 0s 4ms/step - loss: 0.1221 - accuracy: 0.9592 - val_loss: 1.6126 - val_accuracy: 0.7866
Epoch 1308/1500
38/38 [==============================] - 0s 4ms/step - loss: 0.0776 - accuracy: 0.9699 - val_loss: 1.4988 - val_accuracy: 0.7710
Epoch 1309/1500
38/38 [==============================] - 0s 4ms/step - loss: 0.0890 - accuracy: 0.9639 - val_loss: 1.6838 - val_accuracy: 0.7850
Epoch 1310/1500
38/38 [==============================] - 0s 4ms/step - loss: 0.1170 - accuracy: 0.9606 - val_loss: 1.7833 - val_accuracy: 0.7975
Epoch 1311/1500
38/38 [==============================] - 0s 4ms/step - loss: 0.1230 - accuracy: 0.9579 - val_loss: 1.5271 - val_accuracy: 0.7866
Epoch 1312/1500
38/38 [==============================] - 0s 4ms/step - loss: 0.0909 - accuracy: 0.9672 - val_loss: 1.3240 - val_accuracy: 0.7632
Epoch 1313/1500
38/38 [==============================] - 0s 4ms/step - loss: 0.1091 - accuracy: 0.9693 - val_loss: 1.4397 - val_accuracy: 0.7773
E

In [ ]:
# Evaluate the performance
y_pred = model.predict(X_test_reshaped)
y_pred_classes = np.argmax(y_pred, axis=1)
accuracy = accuracy_score(y_test_encoded, y_pred_classes)
print("Accuracy:", accuracy)

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
from pycm import ConfusionMatrix


print("Report : ", classification_report(y_test_encoded, y_pred_classes))

print(ConfusionMatrix(actual_vector=list(y_test_encoded),predict_vector=list(y_pred_classes)))